<img src="../DATA/corp_logo.jpg" width='160'>

# MeanShift
There are a set of data points and we want to assign them into clusters. Imagine these data points are taken, and stack red and blue points on them(the red and blue points overlap completely in the first iteration before starting). The direction of the closest cluster centroid determined by where the most density of data points are at.

So each iteration each blue point will move toward the most dense area(cluster ceneter).

<img src="images/initial_state_meanshift.png" >

#### Iteration 1: 
At the end of iteration 1, all blue points are move towards the clusters.

<img src="images/iteration_1_meanshift.png" >

...

#### Iteration 4:
After subsequent iterations, the cluster means converged.

<img src="images/iteration_4_meanshift.png">

#### Iteration 5:
Identified clusters.

<img src="images/iteration_5_meanshift.png">

- Meanshift can be given a **target** to track, calculate the **histogram** of the target area, then the tracking window keep moving forward to the closest match(cluster center).
- Meanshift **doesn't change the window size** if target moves in depth(**CAMshift** is used to **update** the size of the window).

Check the link below for more information:
https://medium.com/@claudio.vindimian/understanding-and-implementing-the-camshift-object-tracking-algorithm-python-81587c24eda8

# CAMShift Tracking

It starts as the way before. After converging the Meanshift, an update in window size and fitting an eclipse to detect the orientation is performed. Agan, the whole process is repeated unitill some sort of accuracy is met .


<img src='images/meanshift_converged.png'> &nbsp;&nbsp;&nbsp; <img src='images/roi_camshift.png'> &nbsp;&nbsp;&nbsp; <img src='images/orientation_camshift.png'> 
&nbsp;&nbsp;&nbsp;<img src='images/update_meanshift.png'> &nbsp;&nbsp;&nbsp; <img src='images/mean_shift_again.png'>

---
<br>

### MeanShift Tracking

>**Note:**
>
>It is important to start a good position for detecting face for the first frame.

In [1]:
import numpy as np
import cv2 

# Capture a video stream
cap = cv2.VideoCapture(0)

# cap = cv2.VideoCapture()
# cap.open("/dev/v4l/by-id/usb-Sonix_Technology_Co.__Ltd._USB_2.0_Camera-video-index0")

# take first frame of the video
ret,frame = cap.read()


# Set Up the Initial Tracking Window


# We will first detect the face and set that as our starting box.
face_cascade = cv2.CascadeClassifier('../DATA/haarcascades/haarcascade_frontalface_default.xml')
face_rects = face_cascade.detectMultiScale(frame) 

# Convert this list of a single array to a tuple of (x,y,w,h)
(face_x,face_y,w,h) = tuple(face_rects[0]) 
track_window = (face_x,face_y,w,h)
# set up the ROI for tracking
roi = frame[face_y:face_y+h, face_x:face_x+w]


# Use the HSV Color Mapping
hsv_roi =  cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)

# Find histogram to backproject the target on each frame for calculation of meanshit
roi_hist = cv2.calcHist([hsv_roi],[0],None,[180],[0,180])

# Normalize the histogram array values given a min of 0 and max of 255
cv2.normalize(roi_hist,roi_hist,0,255,cv2.NORM_MINMAX)


# Setup the termination criteria, either 10 iteration or move by at least 1 pt
term_crit = ( cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 1 )

while True:
    ret ,frame = cap.read()
    if ret == True:
        
        # Grab the Frame in HSV
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        
        # Calculate the Back Projection based off the roi_hist created earlier
        dst = cv2.calcBackProject([hsv],[0],roi_hist,[0,180],1)
        
        # Apply meanshift to get the new coordinates of the rectangle
        ret, track_window = cv2.meanShift(dst, track_window, term_crit)
        
        # Draw the new rectangle on the image
        x,y,w,h = track_window
        img2 = cv2.rectangle(frame, (x,y), (x+w,y+h), (0,0,255),5)
        
        cv2.imshow('img2',img2)
        
        
        k = cv2.waitKey(1) & 0xff
        if k == 27:
            break
        
    else:
        break
        
cv2.destroyAllWindows()
cap.release()

False
None


### CAMShift Tracking

In [3]:
import numpy as np
import cv2 

# Capture a video stream
# cap = cv2.VideoCapture(0)
cap = cv2.VideoCapture()
cap.open("/dev/v4l/by-id/usb-Sonix_Technology_Co.__Ltd._USB_2.0_Camera-video-index0")


# take first frame of the video
ret,frame = cap.read()


# Set Up the Initial Tracking Window


# We will first detect the face and set that as our starting box.
face_cascade = cv2.CascadeClassifier('../DATA/haarcascades/haarcascade_frontalface_default.xml')
face_rects = face_cascade.detectMultiScale(frame) 

# Convert this list of a single array to a tuple of (x,y,w,h)
(face_x,face_y,w,h) = tuple(face_rects[0]) 
track_window = (face_x,face_y,w,h)
# set up the ROI for tracking
roi = frame[face_y:face_y+h, face_x:face_x+w]


# Use the HSV Color Mapping
hsv_roi =  cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)

# Find histogram to backproject the target on each frame for calculation of meanshit
roi_hist = cv2.calcHist([hsv_roi],[0],None,[180],[0,180])

# Normalize the histogram array values given a min of 0 and max of 255
cv2.normalize(roi_hist,roi_hist,0,255,cv2.NORM_MINMAX)


# Setup the termination criteria, either 10 iteration or move by at least 1 pt
term_crit = ( cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 1 )

while True:
    ret ,frame = cap.read()
    if ret == True:
        
        # Grab the Frame in HSV
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        
        # Calculate the Back Projection based off the roi_hist created earlier
        dst = cv2.calcBackProject([hsv],[0],roi_hist,[0,180],1)
        
        #########################################################
        #########################################################
        ########## CAM SHIFT ####################################
        ########################################################
        #######################################################
        
        # Apply Camshift to get the new coordinates of the rectangle
        ret, track_window = cv2.CamShift(dst, track_window, term_crit)
        
        # Draw it on image
        pts = cv2.boxPoints(ret)
        pts = np.int0(pts)
        img2 = cv2.polylines(frame,[pts],True, (0,0,255),5)
        cv2.imshow('img2',img2)
        
        ########################################################
        #######################################################
        ########################################################
        #######################################################
        
        k = cv2.waitKey(1) & 0xff
        if k == 27:
            break
        
    else:
        break
        
cv2.destroyAllWindows()
cap.release()

IndexError: tuple index out of range